## Train a Tensorflow Model + Build a Flask WebApp + Deploy on Cloud

This post shows and end to end flow of deploying a Machine Learning model on cloud that can predict given a set of new and unseen data. The steps involved are:
- Use Tensorflow to train a model 
- Save the tensorflow model
- Build Web Form using a Flask
- Restore the saved model in the WebApp
- Use the restored model to predict on unseen data
- Deploy the WebApp on cloud using Heroku

### Train a Tensorflow model

Here I use a simple dataset (Auto) from UCI repository and use only a few features of the dataset.

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

**Data preparation i.e. mean centering and normalization**

Taking only cylinders, horsepowerm weight, year and origin as features to predict miles per gallon

In [2]:
auto = pd.read_csv('auto-mpg.data.csv', na_values='?')
auto = auto[['mpg', 'cylinders', 'hp', 'wt', 'year', 'origin']]
auto = pd.get_dummies(auto, columns=['cylinders', 'origin'], drop_first=True)
auto['hp'] = auto['hp'].fillna(auto['hp'].mean(axis=0))
xtrain, xtest, ytrain, ytest = train_test_split(auto.iloc[:,1:10], auto.iloc[:,0], test_size = .2, random_state = 123)

tempAuto = pd.read_csv('auto-mpg.data.csv', na_values='?')
tempAuto = tempAuto.loc[xtest.index]
tempAuto = tempAuto.iloc[:,[0, 1, 3, 4, 6, 7]]
tempAuto.to_csv('/Stuff/Courses/Projects/Flask/Auto/tempAuto.csv')

tMeans = [xtest['hp'].mean(axis=0), xtest['wt'].mean(axis=0), xtest['year'].mean(axis=0)]
tStds = [xtest['hp'].std(axis=0), xtest['wt'].std(axis=0), xtest['year'].std(axis=0)]

xtrain['hp'] = xtrain['hp'] - xtrain['hp'].mean(axis=0)
xtrain['wt'] = xtrain['wt'] - xtrain['wt'].mean(axis=0)
xtrain['year'] = xtrain['year'] - xtrain['year'].mean(axis=0)

xtrain['hp'] = xtrain['hp'] / xtrain['hp'].std()
xtrain['wt'] = xtrain['wt'] / xtrain['wt'].std()
xtrain['year'] = xtrain['year'] / xtrain['year'].std()

xtest['hp'] = xtest['hp'] - xtest['hp'].mean(axis=0)
xtest['wt'] = xtest['wt'] - xtest['wt'].mean(axis=0)
xtest['year'] = xtest['year'] - xtest['year'].mean(axis=0)

xtest['hp'] = xtest['hp'] / xtest['hp'].std()
xtest['wt'] = xtest['wt'] / xtest['wt'].std()
xtest['year'] = xtest['year'] / xtest['year'].std()

xtrain = xtrain.values
ytrain = ytrain.values

xtest = xtest.values
ytest = ytest.values

ytrain.shape = (len(ytrain), 1)
ytest.shape = (len(ytest), 1)

**Build a graph in tensorflow and run the graph**

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 9], name = 'x1')
y = tf.placeholder(tf.float32, shape=[None, 1])

w1 = tf.Variable(tf.random_normal([9, 100], seed=123))
b1 = tf.Variable(tf.zeros([100]))
w2 = tf.Variable(tf.random_normal([100, 1], seed=123))
b2 = tf.Variable(tf.zeros([1]))

h1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
h2 = tf.add(tf.matmul(h1, w2), b2, name='pred')

loss = tf.losses.mean_squared_error(labels=y, predictions=h2)
train_step = tf.train.GradientDescentOptimizer(.01).minimize(loss)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for i in range(2000):
    _, trainLoss = sess.run([train_step, loss], feed_dict = {x:xtrain, y:ytrain})
    testLoss = sess.run(loss, feed_dict={x:xtest, y:ytest})

print('Train Loss:', trainLoss, 'Test Loss:', testLoss)

Train Loss: 5.77966 Test Loss: 8.48845


### Save the model

Save the model in the same session and give it a name to resotre it later

In [4]:
saver = tf.train.Saver()
saver.save(sess, 'auto')

'auto'

### Build a Flask WebApp

The main file of the app. 
The main page shows two text boxes to enter the horsepower and weight values and three dropdowns to select the year, cylinders and origin. After this in the if condition of the index method the predict method of the model file is called which takes in the new data and gives the prediction.

In [ ]:
from flask import Flask, render_template, request
from flask_wtf import FlaskForm
from wtforms import Form, FloatField, SelectField, validators
from flask_bootstrap import Bootstrap
import pandas as pd
import model

app = Flask(__name__)
app.secret_key = 'aaa'
Bootstrap(app)

# Model
class InputForm(FlaskForm):
    horsepower = FloatField('Horse Power', validators=[validators.InputRequired()])
    weight = FloatField('Weight', validators=[validators.InputRequired()])

    year = SelectField('Year', choices=[('70', '1970'), ('71', '1971'), ('72', '1972'), ('73', '1973'), ('74', '1974'), ('75', '1975'),
    ('76', '1976'), ('77', '1977'), ('78', '1978'), ('79', '1979'), ('80', '1980'), ('81', '1981')])

    cylinders = SelectField('Cylinders', choices=[('3', 'Three'), ('4', 'Four'), ('5', 'Five'), ('6', 'Six'), ('8', 'Eight')])

    origin = SelectField('Origin', choices=[('1', 'One'), ('2', 'Two'), ('3', 'Three')])

# View
@app.route('/', methods=['GET', 'POST'])
def index():
    #testData = pd.read_csv('tempAuto.csv')
    form = InputForm()
    allNums = []
    if form.validate_on_submit():
        allNums = [value for fieldname, value in form.data.items()]
        total = model.predict(allNums)
    else:
        total = None

    return render_template("index.html", form=form, data = pd.read_csv('tempAuto.csv').to_html(), total=total)

if __name__ == '__main__':
    app.run(debug=True)

### Restore the saved model

**model.py**

The saved model is restored in this file and the input data is centered and normalized using the values from the test data used in model training.

In [1]:
import tensorflow as tf
import numpy as np

def predict(data):
    data = data[:-1]

    hp = data[0]
    hp -= 100.375
    hp /= 37.948124151033845
    hp = [hp]

    wt = data[1]
    wt -= 2858.7
    wt /= 797.8211055466606
    wt = [wt]

    cylinders = data[2]
    if cylinders == 3:
        cyl = [0,0,0,0]
    elif cylinders == 4:
        cyl = [1,0,0,0]
    elif cylinders == 5:
        cyl = [0,1,0,0]
    elif cylinders == 6:
        cyl = [0,0,1,0]
    else:
        cyl = [0,0,0,1]

    origin = data[3]
    if origin == 1:
        ori = [0,0]
    elif origin == 2:
        ori = [1,0]
    else:
        ori = [0,1]

    year = float(data[4])
    year -= 75.75
    year /= 3.7399654774010815
    year = [year]

    newData = hp + wt + cyl + ori + year
    x = np.array(newData)
    x.shape = (1,9)

    saver = tf.train.import_meta_graph('./auto.meta')
    sess = tf.Session()
    saver.restore(sess, 'auto')

    result = sess.run('pred:0', feed_dict={'x1:0':x})

    return result[0,0]

**index.html**

![](/assets/images/index.png)

### Deploy the App using Heroku

**Login to your Heroku account**

$ heroku login

**change directory to the project directory**

$ cd auto/

**Initialize it as a git repository**

$ git init

**Create Heroku as the remote**

$ heroku git:remote -a auto-predict

**Commit changes and push to the remote**

$ git add .

$ git commit -m "commit message"

$ git push heroku master